# Q.1

In [2]:
from sentence_transformers import SentenceTransformer

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

In [5]:
user_question = "I just discovered the course. Can I still join it?"
embedding = embedding_model.encode(user_question)

In [7]:
embedding[0]

0.078222655

In [8]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

# Q.2

In [21]:
embeddings = []

for doc in documents:
    qa_text = f"{doc['question']} {doc['text']}"
    embedding = embedding_model.encode(qa_text).tolist()
    embeddings.append(embedding)

In [22]:
import numpy as np

X = np.array(embeddings)
X.shape

(948, 768)

# Q.3

In [33]:
v = embedding

In [36]:
scores = X.dot(v)

In [37]:
scores

array([-2.26247554e-04,  1.46293212e-01,  3.36397244e-02, -7.55514930e-03,
        3.30942720e-01,  1.25347289e-01,  2.48137664e-01,  1.37767125e-01,
        7.30194922e-02,  2.93089759e-02, -1.12574597e-02,  6.74266451e-02,
        1.94063765e-02,  2.52825476e-02,  6.40743148e-02, -2.03856430e-02,
        1.30665729e-01,  7.70493156e-03, -3.62687578e-02,  5.13908948e-02,
        1.96568719e-01,  1.91869196e-01,  2.01685225e-01,  2.18906288e-01,
        1.85444111e-01,  7.93294522e-02,  1.14318718e-01,  1.71254438e-01,
        2.38735331e-01,  6.83580309e-02,  5.39974512e-02,  2.03193553e-01,
        5.84990845e-02,  1.03802203e-01,  1.27347372e-01, -1.72261397e-03,
        1.11883599e-01,  6.39170403e-02,  7.77498050e-02,  2.30139904e-01,
        4.48720818e-02,  1.69397091e-01,  1.49489494e-01, -6.46743346e-02,
        1.17909706e-01,  8.84993761e-02, -1.00484974e-02,  1.24248157e-01,
        9.53606455e-02,  8.04818865e-02,  1.23677061e-01,  1.72347088e-01,
        9.86671534e-02,  

In [41]:
np.max(scores)

0.9999999812027365

In [42]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Problem description\nInfrastructure created in AWS with CD-Deploy Action needs to be destroyed\nSolution description\nFrom local:\nterraform init -backend-config="key=mlops-zoomcamp-prod.tfstate" --reconfigure\nterraform destroy --var-file vars/prod.tfvars\nAdded by Erick Calderin',
  'section': 'Module 6: Best practices',
  'question': 'How to destroy infrastructure created via GitHub Actions',
  'course': 'mlops-zoomcamp',
  'id': '886d1617'},
 {'text': 'If you are on the free trial account on GCP you will face this issue when trying to deploy the infrastructures with terraform. This service is not available for this kind of account.\nThe solution I found was to delete the load_balancer.tf file and to comment or delete the rows that differentiate it on the main.tf file. After this just do terraform destroy to delete any infrastructure created on the fail attempts and re-run the terraform apply.\nCode on main.tf to comment/delete:\nLine 166, 167, 168',
  'section': 'Module 